### Bug Hunt: Invisible Shape Broadcasting

This bug  does not have an explicit error and is extremely difficult to find.

Instead, debug defensively!

1. Insert `assert`s for what you think the array sizes should be after every few lines `forward(...)`

2. You can also use `import pdb; pdb.set_trace()` after the first line in `def forward(...)` and step line-by-line, checking the results  of every line.

In [ ]:
!python ../src/download.py

n_item 3953
n_user 6041
n_featuers 9994
n_occu 21
n_rows 1000209


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from torch import from_numpy
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.utils.data import BatchSampler
from torch.utils.data import RandomSampler

fh = np.load('data/dataset.npz')

# We have a bunch of feature columns and last column is the y-target
# Note pytorch is finicky about need int64 types
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']

# We've already split into train & test
X_test = fh['test_x'].astype(np.int64)
Y_test = fh['test_y']

X_train, X_val, Y_train, Y_val = train_test_split(train_x, train_y)


n_user = int(fh['n_user'])
n_item = int(fh['n_item'])

# columns are user_id, item_id and other features 
# we won't use the 3rd and 4th columns
print(X_train)
print(' ')
print(Y_train)



def dataloader(*arrs, batch_size=32):
    dataset = TensorDataset(*arrs)
    bs = BatchSampler(RandomSampler(dataset), 
                      batch_size=batch_size, drop_last=False)
    return DataLoader(dataset, batch_sampler=bs, num_workers=8)
 
train = dataloader(from_numpy(X_train), from_numpy(Y_train))
test = dataloader(from_numpy(X_test), from_numpy(Y_test))
val = dataloader(from_numpy(X_val), from_numpy(Y_val))

[[5763 2942  433    1]
 [2665 2795  116   11]
 [2918 2840  321   12]
 ...
 [4490  353  326    5]
 [1969 1023  173    3]
 [2764 3507  260    7]]
 
[[3.]
 [5.]
 [4.]
 ...
 [3.]
 [4.]
 [5.]]


In [ ]:
from abstract_model import AbstractModel

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl

from pytorch_lightning.loggers import TensorBoardLogger


def l2_regularize(array):
    return torch.sum(array ** 2.0)


class MF(AbstractModel):
    def __init__(self, n_user, n_item, k=18, c_vector=1.0, c_bias=1.0, batch_size=128):
        super().__init__()
        # These are simple hyperparameters
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_vector = c_vector
        self.c_bias = c_bias
        self.batch_size = batch_size
        self.save_hyperparameters()
        
        # These are learned and fit by PyTorch
        self.user = nn.Embedding(n_user, k)
        self.item = nn.Embedding(n_item, k)
        
        # We've added new terms here:
        self.bias_user = nn.Embedding(n_user, 1)
        self.bias_item = nn.Embedding(n_item, 1)
        self.bias = nn.Parameter(torch.ones(1))
    
    def forward(self, inputs):
        # This is the most import function in this script
        # These are the user indices, and correspond to "u" variable
        import pdb; pdb.set_trace()
        user_id = inputs[:, 0]
        # Item indices, correspond to the "i" variable
        item_id = inputs[:, 1]
        # vector user = p_u
        vector_user = self.user(user_id)
        # vector item = q_i
        vector_item = self.item(item_id)
        # this is a dot product & a user-item interaction: p_u * q_i
        ui_interaction = torch.sum(vector_user * vector_item)
        
        # Pull out biases
        bias_user = self.bias_user(user_id)
        bias_item = self.bias_item(item_id)
        biases = (self.bias + bias_user + bias_item)

        # Add bias prediction to the interaction prediction
        prediction = ui_interaction + biases
        return prediction
    
    def loss(self, prediction, target):
        # MSE error between target = R_ui and prediction = p_u * q_i
        loss_mse = F.mse_loss(prediction, target.squeeze())
        return loss_mse, {"mse": loss_mse}
    
    def reg(self):
        # Add new regularization to the biases
        reg_bias_user =  l2_regularize(self.bias_user.weight) * self.c_bias
        reg_bias_item = l2_regularize(self.bias_item.weight) * self.c_bias
        # Compute L2 reularization over user (P) and item (Q) matrices
        reg_user =  l2_regularize(self.user.weight) * self.c_vector
        reg_item = l2_regularize(self.item.weight) * self.c_vector
        # Add up the MSE loss + user & item regularization
        log = {"reg_user": reg_user, "reg_item": reg_item,
               "reg_bias_user": reg_bias_user, "reg_bias_item": reg_bias_item}
        total = reg_user + reg_item + reg_bias_user + reg_bias_item
        return total, log

In [ ]:
from pytorch_lightning.loggers.wandb import WandbLogger

k = 5
c_vector = 1e-5
c_bias = 5e-8
model = MF(n_user, n_item, k=k, c_bias=c_bias, c_vector=c_vector,
          batch_size=1024)

# add a logger
logger = WandbLogger(name="02_mf", project="simple_mf")

trainer = pl.Trainer(max_epochs=100, logger=logger,
                     early_stop_callback=True,
                     progress_bar_refresh_rate=1) 

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
/opt/conda/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)


In [ ]:
trainer.fit(model, train, val)

wandb: Waiting for W&B process to finish, PID 7089
wandb: Program ended successfully.


wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20200925_152916-1gvxdsnv/logs/debug.log
wandb: Find internal logs for this run at: wandb/run-20200925_152916-1gvxdsnv/logs/debug-internal.log
wandb: Run summary:
wandb:     global_step 4399
wandb:             mse 21.899301528930664
wandb:        reg_user 0.28257447481155396
wandb:        reg_item 0.16823159158229828
wandb:   reg_bias_user 0.000289719580905512
wandb:   reg_bias_item 0.000167540623806417
wandb:      train_loss 22.35056495666504
wandb:           epoch 4
wandb:           _step 90
wandb:        _runtime 222
wandb:      _timestamp 1601047980
wandb:        val_loss 5456.27783203125
wandb: Run history:
wandb:     global_step ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
wandb:             mse ▁▂▁▂▁▃▁▁▁▁█▁▁▃▂▂▁▁▁▁▂▁▃▁▁▁▁▂▁▂▁▁▂▂▁▁▂▁▁▁
wandb:        reg_user █▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
wandb:        reg_item █▇▆▅▅▅▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂


  | Name      | Type      | Params
----------------------------------------
0 | user      | Embedding | 30 K  
1 | item      | Embedding | 19 K  
2 | bias_user | Embedding | 6 K   
3 | bias_item | Embedding | 3 K   


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


Saving latest checkpoint..
Epoch 00021: early stopping triggered.


1